Import the libraries

In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### EXTRACT THE SUBCATEGORIES LINKS IN FAQ WEB PAGE

In [42]:
# define the url 
url = r"https://faq.cc.metu.edu.tr"

In [43]:
# define the request 
session_req = requests.session()
response  = session_req.get(url)

In [44]:
# parse the url context
soup = BeautifulSoup(response.text, "html.parser")
# and find the all extensions of the FAQ's links 
network_section = soup.find_all("a",class_='menu__link')

In [45]:
subcategories_links = []
for k in network_section:
    _temp_link = "https://faq.cc.metu.edu.tr" + str(k.get("href"))
    subcategories_links.append(_temp_link)

In [46]:
subcategories_links[:5]

['https://faq.cc.metu.edu.trhttp://faq.cc.metu.edu.tr/tr',
 'https://faq.cc.metu.edu.trhttp://metu.edu.tr/',
 'https://faq.cc.metu.edu.trhttps://faq.cc.metu.edu.tr/faq/cc-faq',
 'https://faq.cc.metu.edu.trNone',
 'https://faq.cc.metu.edu.tr/groups/network']

In [47]:
subcategories_links = subcategories_links[4:]
subcategories_links.remove('https://faq.cc.metu.edu.trhttps://faq.cc.metu.edu.tr/groups/smartcard')
subcategories_links.remove('https://faq.cc.metu.edu.tr/groups/exam-evaluation-optical-reader')
subcategories_links.remove('https://faq.cc.metu.edu.tr/groups/informatics-services')

In [48]:
subcategories_links.append('https://faq.cc.metu.edu.tr/groups/smartcard')

In [50]:
subcategories_links.remove('https://faq.cc.metu.edu.tr/groups/exam-evaluation-optical-reader')

#### NOW GET THE QUESTIONS FROM THE SUBCATEGORIES

In [51]:
FAQs = []
for i in subcategories_links:
    # define the request 
    session_req = requests.session()
    response  = session_req.get(i)

    # parse the url context
    soup = BeautifulSoup(response.text, "html.parser")
    # and find the all extensions of the FAQ's links 
    a = soup.find("table").find_all("a")

    # get the complete links of FAQs
    link_FAQ = []
    for j in a:
        _temp_link = "https://faq.cc.metu.edu.tr/" + j.get("href")
        _title = j.text
        link_FAQ.append((_temp_link,_title))

    # get the questions, answers, and the link
    for k in link_FAQ:
        _temp_link = k[0]
        _temp_title = k[1]
        _temp_response = session_req.get(_temp_link)
        _temp_soup = BeautifulSoup(_temp_response.text,"html.parser")
        _main_body = _temp_soup.find("div", attrs = {"id":"mainbody"})
        _temp_answer = _main_body.text
        FAQs.append({"link": _temp_link,"question": _temp_title, "answer": _temp_answer})


#### DATA PREPROCESSING

In [53]:
len(FAQs)

581

###### Original length of FAQ, so there are 581 questions, and their answers, and their links. However, there might be dublicated questions/answers.

In [69]:
# remove dublicated questions.
unique_faqs = [dict(t) for t in {tuple(d.items()) for d in FAQs}]

In [70]:
len(unique_faqs)

322

In [74]:
# import data as excel

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(FAQs)

df.head(3)

,link,question,answer
0,https://faq.cc.metu.edu.tr//faq/how-can-i-conn...,How can I connect to eduroam wireless network ...,"\n\nIf you are using Windows 10, then you do n..."
1,https://faq.cc.metu.edu.tr//faq/meturoam,How can I connect to meturoam network?,\n\nA device must support WPA2 Enterprise with...
2,https://faq.cc.metu.edu.tr//faq/guest-account-...,A guest account created for me. How can I conn...,\n\nA device must support WPA2 Enterprise with...


In [75]:
df.to_excel(r'C:\Users\merve\Downloads\FAQs_eng.xlsx', index=False)